## Preconfiguration

In [1]:
# install packages
!pip install -q -U keras-nlp datasets
!pip install -q -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00


In [2]:
# configure environment
import os

# operating system environment
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

# training environment
token_length_limit = 256
data_length_limit = 100
lora_name = "cakeboss"
lora_rank = 4
lr_value = 1e-4
train_epoch = 20
model_id = "gemma2_instruct_2b_en"

## Model Initialization

In [3]:
# load Gemma model
import keras
import keras_nlp
import time

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# functions for Gemma model
tick_start = 0
def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"Total time elapsed: {time.time() - tick_start}s")

def text_gen(prompt):
    tick()
    input = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    output = gemma_lm.generate(input, max_length=token_length_limit)
    print("\nGemma Output:")
    print(output)
    tock()

In [5]:
# inference test
text_gen("Hi, nice to meet you! Introduce youself!")


Gemma Output:
<start_of_turn>user
Hi, nice to meet you! Introduce youself!<end_of_turn>
<start_of_turn>model
Hello! It's nice to meet you too. 😊 

I'm Gemma, an AI assistant created by the Gemma team. I'm a large language model, which means I'm really good at understanding and generating text.  

I can help you with things like:

* **Answering your questions:**  I've been trained on a massive amount of text data, so I can provide information on a wide range of topics.
* **Writing different kinds of creative content:**  I can help you write stories, poems, articles, and more.
* **Summarizing text:**  Give me a long piece of writing, and I can give you a concise summary.
* **Translating languages:**  I can translate text between many different languages.

What can I do for you today? 
<end_of_turn>
Total time elapsed: 19.73879384994507s


## Loading Dataset

In [6]:
from datasets import load_dataset

# load example toy dataset
tokenizer = keras_nlp.models.GemmaTokenizer.from_preset(model_id)
data_set = load_dataset(
    "bebechien/korean_cake_boss",
    split="train"
)

print("*" * 50)
print("Dataset insight:")
print(data_set)

README.md:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

**************************************************
Dataset insight:
Dataset({
    features: ['input', 'output'],
    num_rows: 20
})


In [7]:
# format dataset type to Numpy for fast computation
data_set = data_set.with_format("np", columns=["input", "output"], output_all_columns=False)

print("*" * 50)
print("Dataset examples:")
print(data_set[:1])

**************************************************
Dataset examples:
{'input': array(['안녕하세요,\r\n2주 뒤에 있을 아이 생일을 위해 3호 케이크 3개를 주문하고 싶은데 가능할까요?'],
      dtype='<U53'), 'output': array(['고객님, 안녕하세요.\r\n\r\n2주 뒤 아이 생일을 위한 3호 케이크 2개 주문 문의 감사합니다.\r\n네, 3호 케이크 2개 주문 가능합니다.\r\n\r\n아이 생일 케이크인 만큼 더욱 신경 써서 정성껏 준비하겠습니다. 혹시 원하시는 디자인이나 특별한 요청 사항이 있으시면 편하게 말씀해주세요.\r\n\r\n픽업 날짜와 시간을 알려주시면 더욱 자세한 안내를 도와드리겠습니다.\r\n\r\n다시 한번 문의 감사드리며, 아이 생일 진심으로 축하합니다!\r\n\r\n[가게 이름] 드림'],
      dtype='<U249')}


In [8]:
# create training data (strings) using dataset
training_data = []

for row in data_set:
    # training data
    item = f"<start_of_turn>user\n{row['input']}<end_of_turn>\n<start_of_turn>model\n{row['output']}<end_of_turn>"

    # limit the length of tokens of each item
    token_length = len(tokenizer(item))
    if token_length > token_length_limit:
        continue 
    else:
        training_data.append(item) 

    # limit the length of 
    data_length = len(training_data)
    if data_length >= data_length_limit:
            break

In [9]:
# test results 
print("*" * 50)
print("The length of training data:")
print(len(training_data))

print("*" * 50)
print("Example:")
print(training_data[0])

**************************************************
The length of training data:
11
**************************************************
Example:
<start_of_turn>user
안녕하세요,
2주 뒤에 있을 아이 생일을 위해 3호 케이크 3개를 주문하고 싶은데 가능할까요?<end_of_turn>
<start_of_turn>model
고객님, 안녕하세요.

2주 뒤 아이 생일을 위한 3호 케이크 2개 주문 문의 감사합니다.
네, 3호 케이크 2개 주문 가능합니다.

아이 생일 케이크인 만큼 더욱 신경 써서 정성껏 준비하겠습니다. 혹시 원하시는 디자인이나 특별한 요청 사항이 있으시면 편하게 말씀해주세요.

픽업 날짜와 시간을 알려주시면 더욱 자세한 안내를 도와드리겠습니다.

다시 한번 문의 감사드리며, 아이 생일 진심으로 축하합니다!

[가게 이름] 드림<end_of_turn>


## LoRA Fine-tuning